In [ ]:

# 1. INSTALASI LIBRARY YANG DIPERLUKAN
# ===================================================================
!pip install transformers datasets torch accelerate sentencepiece -q


In [ ]:
import torch
import pandas as pd
import numpy as np
from transformers import (
    ElectraTokenizer,
    ElectraForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset, load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import warnings
warnings.filterwarnings('ignore')

In [ ]:
MODEL_NAME = "ChristopherA08/IndoELECTRA"  # Model ELECTRA Bahasa Indonesia
MAX_LENGTH = 128
BATCH_SIZE = 64
EPOCHS = 3
LEARNING_RATE = 2e-5

# Cek GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device yang digunakan: {device}")

Device yang digunakan: cuda


In [ ]:
# 4. MOUNT GOOGLE DRIVE DAN LOAD DATASET
# ===================================================================
from google.colab import drive
drive.mount('/content/drive')

print("Google Drive berhasil di-mount!")
print("\n" + "="*50)

# GANTI PATH INI SESUAI LOKASI FILE ANDA DI GOOGLE DRIVE
# Contoh format path:
# - '/content/drive/MyDrive/dataset_bukalapak.csv'
# - '/content/drive/MyDrive/Folder Dataset/bukalapak.json'
# - '/content/drive/MyDrive/bukalapak_data.xlsx'

FILE_PATH = '/content/train.csv'  # ⬅️ GANTI INI

print(f"Membaca dataset dari: {FILE_PATH}")

# Deteksi format file dan load sesuai ekstensi
import os
file_extension = os.path.splitext(FILE_PATH)[1].lower()

if file_extension == '.csv':
    df = pd.read_csv(FILE_PATH)
    print("✓ File CSV berhasil dimuat")
elif file_extension == '.json':
    df = pd.read_json(FILE_PATH)
    print("✓ File JSON berhasil dimuat")
elif file_extension in ['.xlsx', '.xls']:
    df = pd.read_excel(FILE_PATH)
    print("✓ File Excel berhasil dimuat")
else:
    raise ValueError(f"Format file tidak didukung: {file_extension}")

# Tampilkan informasi dataset
print(f"\nJumlah data: {len(df)}")
print(f"Kolom yang tersedia: {df.columns.tolist()}")
print(f"\nContoh 3 data pertama:")
print(df.head(3))

# PASTIKAN DATASET MEMILIKI KOLOM 'text' DAN 'label'
# Jika nama kolom berbeda, rename di sini:
# df = df.rename(columns={'deskripsi': 'text', 'kategori': 'label'})

# Bersihkan data: hapus NaN/null
print(f"\nJumlah data sebelum cleaning: {len(df)}")
df = df.dropna(subset=['review_sangat_singkat', 'label'])
print(f"Jumlah data setelah cleaning: {len(df)}")

# Konversi label ke numerik jika masih berupa string
label_encoder = None
if df['label'].dtype == 'object':
    from sklearn.preprocessing import LabelEncoder
    label_encoder = LabelEncoder()
    df['label'] = label_encoder.fit_transform(df['label'])
    print(f"\n✓ Label dikonversi ke numerik")
    print(f"Mapping label: {dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))}")

    # Simpan label encoder untuk nanti
    import pickle
    with open('label_encoder.pkl', 'wb') as f:
        pickle.dump(label_encoder, f)
else:
    # Pastikan label mulai dari 0 dan berurutan
    unique_labels = sorted(df['label'].unique())
    print(f"\n✓ Label unik yang ditemukan: {unique_labels}")

    # Remap label ke 0, 1, 2, 3, ... jika belum berurutan
    if unique_labels != list(range(len(unique_labels))):
        print("⚠️ Label tidak berurutan dari 0, melakukan remapping...")
        label_map = {old_label: new_label for new_label, old_label in enumerate(unique_labels)}
        df['label'] = df['label'].map(label_map)
        print(f"✓ Label mapping: {label_map}")

# Pastikan label adalah integer
df['label'] = df['label'].astype(int)

# Validasi label range
min_label = df['label'].min()
max_label = df['label'].max()
num_unique_labels = df['label'].nunique()

print(f"\nValidasi label:")
print(f"- Min label: {min_label}")
print(f"- Max label: {max_label}")
print(f"- Jumlah label unik: {num_unique_labels}")
print(f"- Distribusi label:\n{df['label'].value_counts().sort_index()}")

# CRITICAL: Pastikan label mulai dari 0
if min_label != 0:
    raise ValueError(f"Label harus mulai dari 0, tetapi min_label = {min_label}")

# Konversi ke Hugging Face Dataset
dataset = Dataset.from_pandas(df[['review_sangat_singkat', 'label']])  # Ambil hanya kolom yang diperlukan
print(f"\n✓ Dataset berhasil dikonversi ke format Hugging Face")
print(f"Jumlah data: {len(dataset)}")
print(f"Contoh data: {dataset[0]}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive berhasil di-mount!

Membaca dataset dari: /content/train.csv
✓ File CSV berhasil dimuat

Jumlah data: 96329
Kolom yang tersedia: ['id', 'header_review', 'review_sangat_singkat', 'label']

Contoh 3 data pertama:
   id                 header_review  \
0   0  Mantap Barang Sesuai Pesanan   
1   1                  Mantaps.....   
2   2                 Barang Sesuai   

                               review_sangat_singkat  label  
0                       mantap barang sesuai pesanan      1  
1                                       Mantaps.....      1  
2  Terima kasih buka lapaklapak barang sesuai den...      1  

Jumlah data sebelum cleaning: 96329
Jumlah data setelah cleaning: 96329

✓ Label unik yang ditemukan: [np.int64(0), np.int64(1)]

Validasi label:
- Min label: 0
- Max label: 1
- Jumlah label unik: 2
- Distribusi label:
label
0    12211
1    

In [ ]:
# 5. SPLIT DATA: TRAIN, VALIDATION, TEST
# ===================================================================
# Split 80% train, 10% validation, 10% test
train_val = dataset.train_test_split(test_size=0.2, seed=42)
val_test = train_val['test'].train_test_split(test_size=0.5, seed=42)

train_dataset = train_val['train']
val_dataset = val_test['train']
test_dataset = val_test['test']

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

Train: 77063, Val: 9633, Test: 9633


In [ ]:
# 6. LOAD TOKENIZER DAN MODEL
# ===================================================================
from transformers import AutoTokenizer, AutoModelForSequenceClassification

print(f"Loading model: {MODEL_NAME}")

# Load tokenizer dengan error handling
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
except:
    print("⚠️ Fast tokenizer gagal, mencoba slow tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

# Tambahkan special tokens jika belum ada
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("✓ Pad token diset ke EOS token")

print(f"✓ Tokenizer loaded")
print(f"  - Vocab size: {tokenizer.vocab_size}")
print(f"  - Model max length: {tokenizer.model_max_length}")
print(f"  - Pad token: {tokenizer.pad_token}")
print(f"  - Pad token ID: {tokenizer.pad_token_id}")

# Hitung jumlah label unik DARI DATASET
num_labels = int(df['label'].nunique())
print(f"\n{'='*50}")
print(f"KONFIGURASI MODEL")
print(f"{'='*50}")
print(f"Model: {MODEL_NAME}")
print(f"Jumlah kategori/label: {num_labels}")
print(f"Label range: 0 sampai {num_labels - 1}")
print(f"{'='*50}\n")

# Load model dengan jumlah label yang benar
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    ignore_mismatched_sizes=True  # Penting untuk fine-tuning
)

# Resize token embeddings jika diperlukan
model.resize_token_embeddings(len(tokenizer))

print("✓ Model dan tokenizer berhasil dimuat")

Loading model: ChristopherA08/IndoELECTRA
✓ Tokenizer loaded
  - Vocab size: 35000
  - Model max length: 1000000000000000019884624838656
  - Pad token: [PAD]
  - Pad token ID: 0

KONFIGURASI MODEL
Model: ChristopherA08/IndoELECTRA
Jumlah kategori/label: 2
Label range: 0 sampai 1



Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at ChristopherA08/IndoELECTRA and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ Model dan tokenizer berhasil dimuat


In [ ]:
# 7. PREPROCESSING DAN TOKENISASI
# ===================================================================
def preprocess_function(examples):
    """
    Fungsi untuk tokenisasi teks
    """
    # Tokenisasi
    tokenized = tokenizer(
        examples['review_sangat_singkat'],
        truncation=True,
        padding='max_length',
        max_length=MAX_LENGTH
    )

    # Pastikan label dalam format yang benar
    tokenized['labels'] = examples['label']

    return tokenized

print("Melakukan tokenisasi dataset...")
# Tokenisasi semua dataset
tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names  # Hapus kolom asli
)
tokenized_val = val_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=val_dataset.column_names
)
tokenized_test = test_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=test_dataset.column_names
)

print("✓ Tokenisasi selesai")
print(f"Format tokenized_train: {tokenized_train}")

# Data collator untuk dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Melakukan tokenisasi dataset...


Map:   0%|          | 0/77063 [00:00<?, ? examples/s]

Map:   0%|          | 0/9633 [00:00<?, ? examples/s]

Map:   0%|          | 0/9633 [00:00<?, ? examples/s]

✓ Tokenisasi selesai
Format tokenized_train: Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 77063
})


In [ ]:
# 8. METRICS UNTUK EVALUASI
# ===================================================================
def compute_metrics(eval_pred):
    """
    Fungsi untuk menghitung metrik evaluasi
    """
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels,
        predictions,
        average='weighted'
    )

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
# 9. TRAINING ARGUMENTS
# ===================================================================
training_args = TrainingArguments(
    output_dir='./results',                    # Folder output
    num_train_epochs=EPOCHS,                   # Jumlah epoch
    per_device_train_batch_size=BATCH_SIZE,    # Batch size training
    per_device_eval_batch_size=BATCH_SIZE,     # Batch size evaluasi
    learning_rate=LEARNING_RATE,               # Learning rate
    warmup_steps=500,                          # Warmup steps
    weight_decay=0.01,                         # Weight decay
    logging_dir='./logs',                      # Folder logs
    logging_steps=50,                          # Log setiap 50 steps
    eval_strategy="epoch",               # Evaluasi setiap epoch
    save_strategy="epoch",                     # Save model setiap epoch
    load_best_model_at_end=True,              # Load model terbaik
    metric_for_best_model="f1",               # Metrik untuk best model
    push_to_hub=False,                        # Jangan push ke Hugging Face Hub
    report_to="none",
    fp16=True,                                # Mixed precision training (WAJIB!)
    dataloader_num_workers=2,                 # Parallel data loading
    gradient_accumulation_steps=1,            # Gradient accumulation
    save_total_limit=2,                       # Hanya simpan 2 checkpoint terakhir
    disable_tqdm=False,                       # Show progress bar
    # Optimasi tambahan untuk T4 GPU:
    gradient_checkpointing=False,             # False untuk kecepatan
    optim="adamw_torch",  # Disable wandb/tensorboard
)

In [ ]:
# 10. INISIALISASI TRAINER
# ===================================================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
# 11. MULAI TRAINING
# ===================================================================
print("\n" + "="*50)
print("MEMULAI TRAINING...")
print("="*50 + "\n")

trainer.train()


MEMULAI TRAINING...



Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.176600,0.176286,0.931070,0.927800,0.927224,0.931070
2,0.150200,0.178179,0.931901,0.930473,0.929601,0.931901
3,0.131000,0.200506,0.928994,0.927267,0.926272,0.928994


TrainOutput(global_step=3615, training_loss=0.16106117526855046, metrics={'train_runtime': 1541.4357, 'train_samples_per_second': 149.983, 'train_steps_per_second': 2.345, 'total_flos': 1.520709544440576e+16, 'train_loss': 0.16106117526855046, 'epoch': 3.0})

In [ ]:
# 12. EVALUASI PADA TEST SET
# ===================================================================
print("\n" + "="*50)
print("EVALUASI PADA TEST SET")
print("="*50 + "\n")

test_results = trainer.evaluate(tokenized_test)
print(f"Test Results: {test_results}")


EVALUASI PADA TEST SET



Test Results: {'eval_loss': 0.18130148947238922, 'eval_accuracy': 0.9280597944565556, 'eval_f1': 0.9273570470856604, 'eval_precision': 0.9267541724714056, 'eval_recall': 0.9280597944565556, 'eval_runtime': 17.7515, 'eval_samples_per_second': 542.657, 'eval_steps_per_second': 8.506, 'epoch': 3.0}


In [ ]:
# 13. SIMPAN MODEL DAN TOKENIZER
# ===================================================================
model.save_pretrained('./electra_bukalapak_model')
tokenizer.save_pretrained('./electra_bukalapak_model')
print("\nModel dan tokenizer berhasil disimpan di './electra_bukalapak_model'")


Model dan tokenizer berhasil disimpan di './electra_bukalapak_model'


In [ ]:
# 14. FUNGSI UNTUK PREDIKSI
# ===================================================================
def predict_text(text, model, tokenizer):
    """
    Fungsi untuk memprediksi kategori dari teks
    """
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=MAX_LENGTH
    ).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(predictions, dim=-1).item()
        confidence = predictions[0][predicted_class].item()

    return predicted_class, confidence

In [ ]:
# 15. CONTOH PENGGUNAAN PREDIKSI
# ===================================================================
print("\n" + "="*50)
print("CONTOH PREDIKSI")
print("="*50 + "\n")

test_texts = [
    "Sepatu olahraga Nike Air Max bagus banget",
    "Laptop Dell Inspiron 14 inch SSD 512GB nya b aja",
    "Kalau ga niat jualan ga usah jualan disini"
]

model.to(device)
for text in test_texts:
    pred_label, confidence = predict_text(text, model, tokenizer)
    print(f"Text: {text}")
    print(f"Predicted Label: {pred_label}")
    print(f"Confidence: {confidence:.4f}")
    print("-" * 50)


CONTOH PREDIKSI

Text: Sepatu olahraga Nike Air Max bagus banget
Predicted Label: 1
Confidence: 0.9983
--------------------------------------------------
Text: Laptop Dell Inspiron 14 inch SSD 512GB nya b aja
Predicted Label: 1
Confidence: 0.7731
--------------------------------------------------
Text: Kalau ga niat jualan ga usah jualan disini
Predicted Label: 0
Confidence: 0.8764
--------------------------------------------------
